<a href="https://colab.research.google.com/github/prabhat2605/Weed_detection/blob/main/Weed_detection_using_Tfod_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-gpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pwd 


In [ ]:
import tensorflow as tf
print(tf.__version__)

#Cloning TFod 2.0


In [ ]:
!git clone https://github.com/tensorflow/models.git

# New Section

In [ ]:
pwd

In [ ]:
cd /content/drive/MyDrive/TFOD2


In [ ]:
cd /content/drive/MyDrive/TFOD2/models/research

In [ ]:
pwd

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.


In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git


In [ ]:
cd /content/drive/MyDrive/TFOD2/models/research/cocoapi/PythonAPI

In [ ]:
!make

In [ ]:
cp -r pycocotools /content/drive/MyDrive/TFOD2/models/research

#Install the Object Detection API

In [1]:
cd /content/drive/MyDrive/TFOD2/models/research

/content/drive/MyDrive/TFOD2/models/research


In [2]:
pwd

'/content/drive/MyDrive/TFOD2/models/research'

In [3]:
cp object_detection/packages/tf2/setup.py .

In [ ]:
!python -m pip install --use-feature=in-tree-build .

In [5]:
pwd


'/content/drive/MyDrive/TFOD2/models/research'

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

In [7]:
cd /content/drive/MyDrive/TFOD2/training_demo/pre-trained-models

/content/drive/MyDrive/TFOD2/training_demo/pre-trained-models


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz

In [9]:
!tar -xvf ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz

ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/checkpoint/
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/pipeline.config
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/saved_model.pb
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/assets/
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.index


In [10]:
pwd 

'/content/drive/MyDrive/TFOD2/training_demo/pre-trained-models'

In [11]:
cd /content/drive/MyDrive/TFOD2/training_demo

/content/drive/MyDrive/TFOD2/training_demo


In [ ]:
# Create train data:
!python generate_tfrecord.py -x /content/drive/MyDrive/TFOD2/training_demo/images/train -l /content/drive/MyDrive/TFOD2/training_demo/annotations/label_map.pbtxt -o /content/drive/MyDrive/TFOD2/training_demo/annotations/train.record

# Create test data:
!python generate_tfrecord.py -x /content/drive/MyDrive/TFOD2/training_demo/images/test -l /content/drive/MyDrive/TFOD2/training_demo/annotations/label_map.pbtxt -o /content/drive/MyDrive/TFOD2/training_demo/annotations/test.record

In [ ]:
pwd

In [ ]:
cd /content/drive/MyDrive/TFOD2/training_demo

In [ ]:
!python model_main_tf2.py --model_dir=/content/drive/MyDrive/TFOD2/training_demo/models/my_model_resnet101_v1_fpn --pipeline_config_path=/content/drive/MyDrive/TFOD2/training_demo/models/my_ssd_resnet101_v1_fpn/pipeline.config

In [ ]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python


In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path /content/drive/MyDrive/TFOD2/training_demo/models/my_ssd_resnet101_v1_fpn/pipeline.config --trained_checkpoint_dir /content/drive/MyDrive/TFOD2/training_demo/models/my_model_resnet101_v1_fpn --output_directory /content/drive/MyDrive/TFOD2/training_demo/exported_model/my_model

Infrencing my trained model

In [ ]:
"""
Object Detection (On Image) From TF2 Saved Model
=====================================
"""

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import cv2
import argparse
from google.colab.patches import cv2_imshow

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# PROVIDE PATH TO IMAGE DIRECTORY
IMAGE_PATHS = '/content/drive/MyDrive/TFOD2/training_demo/images/test/189.jpg'


# PROVIDE PATH TO MODEL DIRECTORY
PATH_TO_MODEL_DIR = '/content/drive/MyDrive/TFOD2/training_demo/exported_model/my_model'

# PROVIDE PATH TO LABEL MAP
PATH_TO_LABELS = '/content/drive/MyDrive/TFOD2/training_demo/annotations/label_map.pbtxt'

# PROVIDE THE MINIMUM CONFIDENCE THRESHOLD
MIN_CONF_THRESH = float(0.60)

# LOAD THE MODEL

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# LOAD SAVED MODEL AND BUILD DETECTION FUNCTION
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# LOAD LABEL MAP DATA FOR PLOTTING

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.
    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.
    Args:
      path: the file path to the image
    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))




print('Running inference for {}... '.format(IMAGE_PATHS), end='')

image = cv2.imread(IMAGE_PATHS)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_expanded = np.expand_dims(image_rgb, axis=0)

# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]

# input_tensor = np.expand_dims(image_np, 0)
detections = detect_fn(input_tensor)

# All outputs are batches tensors.
# Convert to numpy arrays, and take index [0] to remove the batch dimension.
# We're only interested in the first num_detections.
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
               for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_with_detections = image.copy()

# SET MIN_SCORE_THRESH BASED ON YOU MINIMUM THRESHOLD FOR DETECTIONS
viz_utils.visualize_boxes_and_labels_on_image_array(
      image_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=0.5,
      agnostic_mode=False)

print('Done')
# DISPLAYS OUTPUT IMAGE
cv2_imshow(image_with_detections)
# CLOSES WINDOW ONCE KEY IS PRESSED
